In [1]:
from tqdm import tqdm
import pandas as pd
import json
import requests

In [2]:
bot_name = 'curekart_subset'
auth = 'ya29.c.Ko8B2AePTyGL92WW5KdFoL6J-tcsS9Nece3p-A-YD7I7yL-lzJV9sAJY3lnvOLpvtor4bmE5AiXHVRCWdvWeuE6oyY80EwFpPVXwa892Pjsx4-WOQl5xSv_GWYH9rLUeCT87KR2Yr2zKpO5SnXikRSqy-UHJRobWcVJzoURBx-HXtG7HrsK1bIEUWjsZT9rAgTA'
url = 'https://dialogflow.clients6.google.com/v2/projects/curekart-subset-kooi/agent/sessions/66b129c7-a06d-81b5-9ba1-a255a4922bed:detectIntent'

In [3]:
def predict_node_name(text, url, auth):
    headers = {
        'Content-Type': 'application/json; charset=utf-8',
        'Authorization': f'Bearer {auth}',
    }
    data = {
        "queryInput":
        {
            "text":{
                "text": text,
                "languageCode":"en"
            }
        },
        "queryParams":{
            "timeZone":"Asia/Calcutta"
            }
        }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if 'intent' in response.json()['queryResult']:
        predicted_node = response.json()['queryResult']['intent']['displayName']
        predicted_node_score = response.json()['queryResult']['intentDetectionConfidence']
    else:
        predicted_node = 'NO_NODES_DETECTED'
        predicted_node_score = 0
    return predicted_node, predicted_node_score

In [4]:
test_file_name = bot_name if '_subset' not in bot_name else bot_name.replace('_subset', '')
df_test = pd.read_csv(f'../../test/{test_file_name}_test.csv')
df_test.head()

,sentence,label
0,Order my product,ORDER_TAKING
1,Hy,NO_NODES_DETECTED
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT
3,have u started C D provision for pin code 702164,CHECK_PINCODE
4,How do I know it will deliver in my place,NO_NODES_DETECTED


In [5]:
df_test.shape

(991, 2)

In [6]:
predict_node_name('Order my product', url, auth)

('ORDER_TAKING', 0.5751104)

In [7]:
pred_nodes = []
pred_scores = []
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    if len(row['sentence']) > 256:
        # this constraint exists on dialogflow
        print(f"Truncating sentence: {row['sentence']}")
        sentence = row['sentence'][:255]
    else:
        sentence = row['sentence']
    node_name, node_score = predict_node_name(sentence, url, auth)
    pred_nodes.append(node_name)
    pred_scores.append(node_score)
df_test['predicted_node'] = pred_nodes
df_test['predicted_node_score'] = pred_scores
df_test.head()

100%|██████████| 991/991 [07:05<00:00,  2.33it/s]


,sentence,label,predicted_node,predicted_node_score
0,Order my product,ORDER_TAKING,ORDER_TAKING,0.575110
1,Hy,NO_NODES_DETECTED,NO_NODES_DETECTED,0.000000
2,I want to order Wottle sensitive handwash refill,RECOMMEND_PRODUCT,ORDER_TAKING,0.542082
3,have u started C D provision for pin code 702164,CHECK_PINCODE,CHECK_PINCODE,0.453903
4,How do I know it will deliver in my place,NO_NODES_DETECTED,RESUME_DELIVERY,0.685046


In [8]:
set(df_test['predicted_node_score'])

{0.0,
 0.20732097,
 0.20739603,
 0.250861,
 0.25444493,
 0.25544554,
 0.2651417,
 0.2669851,
 0.27001488,
 0.27195868,
 0.28805944,
 0.2912983,
 0.29753673,
 0.30113474,
 0.30323094,
 0.30568174,
 0.31007707,
 0.31020167,
 0.31246895,
 0.31396055,
 0.31571227,
 0.31955937,
 0.31994936,
 0.32062307,
 0.32094556,
 0.3233829,
 0.32615197,
 0.32617027,
 0.32662645,
 0.32794368,
 0.3300313,
 0.33012813,
 0.33364335,
 0.33451143,
 0.33490264,
 0.33534628,
 0.3376866,
 0.33781984,
 0.34222952,
 0.34344807,
 0.34662104,
 0.3467206,
 0.34918353,
 0.34979638,
 0.34984174,
 0.3518774,
 0.35198066,
 0.3522799,
 0.35343748,
 0.3566927,
 0.35792378,
 0.35999566,
 0.3618582,
 0.3619326,
 0.36220297,
 0.36237952,
 0.36289003,
 0.36392018,
 0.36598086,
 0.36683595,
 0.36732948,
 0.3687623,
 0.36969683,
 0.3708923,
 0.37483224,
 0.37720895,
 0.37774885,
 0.37936574,
 0.37983292,
 0.38147053,
 0.38192588,
 0.38279492,
 0.38311577,
 0.3835015,
 0.3837254,
 0.38443694,
 0.3853668,
 0.387527,
 0.38871777,
 

In [9]:
set(df_test['predicted_node']) - set(df_test['label'])

{'IMMUNITY',
 'INTERNATIONAL_SHIPPING',
 'MODES_OF_PAYMENTS',
 'PORTAL_ISSUE',
 'REFER_EARN',
 'START_OVER',
 'WORK_FROM_HOME'}

In [10]:
df_test.to_csv(f'../../preds/dialogflow_{bot_name}.csv', index=False)